# ```pandas```

### Voir : https://www.dataquest.io/blog/settingwithcopywarning/

Les trois principales strucutres de données pandas (Series, DataFrame et Panel) sont subsettables à l'aide de leurs index. Subsetter un objet avec un seul élément d'index revoie un objet de la dimension inférieure : subsetter une Series renvoie un scalaire (une liste ? un array ?), subsetter un DataFrame renvoie une Series, etc. Sinon on reste sur le même type d'objet. 

Attention : 
```df.loc[:,'a']``` retourne une Series (single label)
```df.loc[:,['a']]``` retourne un DataFrame (list of labels)

Trois principales méthodes sont à la disposition de l'utilisateur. Chacune jusqu'à un indexer (appelé aussi *axe accessor*) par dimension en argument : 
* ```loc``` qui permet une sélection à l'aide de labels
* ```iloc``` qui permet une sélection à l'aide d'entiers
* ```[]``` qui correspond en fait à un alias pour ```__getitem__```. Comment se situe-t-il par rapport aux deux autres.

On peut globalement passer aux deux premiers : soit un label simple ou une liste de labels (respectivement d'entiers), un objet slice de labels (respectivement d'entiers), un tableau de booléens ou depuis pandas 0.18, un objet callable prenant en argument l'objet qui l'appelle (le DataFrame, la Series, etc.) et retournant un des types d'objets précités.

En particulier, la slice nulle ```:``` est très utilisée pour signifier qu'on souhaite garder l'ensemble d'une dimension. Ex : ```df.iloc[1,:]```

Pour un DataFrame, le subsetting s'écrira : ```df[indexer, indexer]``` ou ```df.loc[indexer, indexer]``` ou ```df.iloc[indexer, indexer]```.

Remarque : ```loc``` et ```iloc``` sont en fait des attributs de l'objet Series / DataFrame / Panel.

Autres attributs utiles pour les DataFrames: 
* ```index``` : retourne les labels des lignes (l'index) (objet pandas.core.indexes.range.RangeIndex)
* ```columns``` : retourne les labels des colonnes (objet pandas.core.indexes.range.RangeIndex). Si on veut juste la liste des noms de colonne, utiliser simplement ```list(df)```.
* ```dtypes``` : retourne des colonnes du DataFrame sous la forme d'une Series indexée par les noms (labels) des colonnes.
* ```shape``` : retourne un tuple rassemblant le nombre de lignes et de colonnes. 
* ```values``` : retourne une représentation numpy du DataFrame (numpy.ndarray)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'a' : [0,1,2,3], 'b' : [0.0,1.5,2.3,3.2], 'c' : ['f','e','s','v']})
df

,a,b,c
0,0,0.0,f
1,1,1.5,e
2,2,2.3,s
3,3,3.2,v


### Entier simple, liste d'entiers ou slice d'entiers

In [3]:
df.iloc[:,1] # renvoie une (pandas.core.series.) Series

0    0.0
1    1.5
2    2.3
3    3.2
Name: b, dtype: float64

In [4]:
df.iloc[:,[0,2]] # renvoie un (pandas.core.frame) DataFrame
# Attention / Rappel : l'indexation des lignes comme des colonnes commence à 0

,a,c
0,0,f
1,1,e
2,2,s
3,3,v


In [23]:
df.iloc[1] # renvoie une (pandas.core.series.) Series
# indentique à 
df.iloc[1,:]
# df.iloc[,1] renvoie en revanche une erreur et n'est donc pas équivalent à df.iloc[:,1]
# df.iloc[1,] est en revanche et de façon surprenante (on écrit jamais f(a,) pour ne rien passer au second argument,
# on écrit simplement f(a)) valide et identique à df.iloc[1,:]

a      1
b    1.5
c      e
Name: 1, dtype: object

In [24]:
df.iloc[1,2] # renvoie un type élémentaire : ce cas particulier correpond au subetting d'un DataFrame puis d'une Series

str

In [42]:
df.iloc[1,:]

a      1
b    1.5
c      e
Name: 1, dtype: object

### Array de booléens

In [85]:
# df.iloc[df['a']>1] not ok car df.iloc[pd.Series([False, False, True, True])] not ok
# df.iloc[[False, False, True, True]] ok
# df[df['d']>1] ok
# df.iloc[np.asarray([False, False, True, True])] ok
# df.iloc[pd.Series([False, False, True, True]).values] ok
# df.iloc[(df['d']>1).values] ok mais moche
# df.loc[(df['d']>1)] ok mais pas pour iloc !!

,a,b,c,d,e
2,12,2.3,s,2,s12
3,13,3.2,v,3,v13


Dataframe : 
iloc
* selections sur les lignes : integer, list of integers, integer slice, boolean array mais pas Series, callable
* selection sur les colonnes : idem ci-dessus (à confirmer sur les booléens)

loc 
* selections sur les lignes : label, list of labels, label slice, boolean array ou Series, callable
* selections sur les colonnes : label, list of labels, label slice, boolean array ou Series, callable (à confirmer sur les booléens)
[]

 

In [83]:
df['e'] = df['c'] + df['a'].astype(str)
df

,a,b,c,d,e
0,10,0.0,f,0,f10
1,11,1.5,e,1,e11
2,12,2.3,s,2,s12
3,13,3.2,v,3,v13


### Indexes comme attributs
S'il est un nom Python valide et qu'il ne rentre pas en conflit avec le nom d'autres attributs ou méthodes, le label d'un index d'une Serie ou le nom de colonne d'un DataFrame (ou le nom d'un item d'un Panel) est ajouté aux attributs de l'objet. On peut alors accéder directement à l'objet sous-jascent.

Remarque : pour un DataFrame par exemple, les expressions suivantes renverront une erreur : ```df.1``` ou ```df.min``` car les noms de colonne '1' et 'min' ne consitue pas un identifiant Python valide et crée un conflit respectivement.

In [45]:
dfp = pd.DataFrame({'a' : [0,1,2,3], 'b' : [0.0,1.5,2.3,3.2], 'c' : ['f','e','s','v']}, index = ['w', 'x', 'y', 'z'])
dfp

,a,b,c
w,0,0.0,f
x,1,1.5,e
y,2,2.3,s
z,3,3.2,v


In [48]:
# dfp possède trois attributs supplémentaire permettant d'accéder à chacune de ses colonnes.
dfp.a # renvoie une pandas.core.series.Series

w    0
x    1
y    2
z    3
Name: a, dtype: int64

```dfp.w``` renvoie une erreur car w est un index de ligne et l'identifiant correspondant n'est pas ajouté aux attributs du DataFrame 

On peut utiliser ces nouveaux attributs pour accéder à et modifier une colonne existante mais pas pour en créer une nouvelle.

In [58]:
df.a
df.a = range(10,14)
# mais df.d = range(3) par exemple va renvoyer une erreur car la colonne d n'existe pas encore, utiliser la forme suivante ou 
# des formes équivalentes 
df['d'] = range(4)
df

,a,b,c,d
0,10,0.0,f,0
1,11,1.5,e,1
2,12,2.3,s,2
3,13,3.2,v,3


Remarque : il existe aussi ```.ix``` mais qui est dépréciée. 

## ```[ ]``` (alias de ```__getitem__```)

Semble dédier à sélectionner une slice de dimension inférieure : 
* Pour une Series : serie[label] qui va retourner un type simple
* Pour un DataFrame : frame[column] qui va retourner une Series
* Pour un Panel : panel[item] qui va retourner un DataFrame

Dans le cas des DataFrame, on peut lui passer un nom (label de colonne) ce qui nous retournera alors une Series, une liste de noms de colonnes qui nous retournera alors un Dataframe. ```[ ]``` accepte également un *callable* (prenant en arguement la Series ou le DataFrame) et retournant un des objets précédemment cités.

```[ ]``` accepte également des slices mais portant uniquement sur l'index (on peut donc slicer avec des entiers mais sans doute aussi avec des datetime) : 
* Dans le cas d'une Series, on récupère une portion de la Series initiale
* Dans le cas d'un DataFrame, on ne sélectionne que sur les lignes et récupère un DataFrame de même nombre de colonnes. Il ne semble pas possible de procéder simultanément à une sélection sur les lignes et les colonnes à l'aide de ```[ ]``` (qui ne semble prendre qu'un argument, à vérifier sur __getitem__). df[0:2, 'a'] renvoie une erreur. 

Contrairement à ```.loc``` et ```.iloc```, ```[ ]``` n'accepte **pas** comme arguments : 
* Des entiers seuls ou des listes d'entiers
* Des arrays de booléens
* Des slices

Comment on sélectionne simultanément sur les colonnes et les lignes ? => loc au moins, pas possible avec []
Application : Comment on filtre ?

In [46]:
ser = df['a']
df[['a', 'a', 'b']]
df[lambda x : ['a']]
df[0:2]
df.loc[df['a'] > 0, 'a']
df[0:2]
df.iloc[0:2]
# df.loc['a'] ne marche pas (assignation positionnelle de l'argument à la première dimension et 'a' n'est pas un label de l'index)
# pour bien lui faire comprendre qu'on veut la colonne : df.loc[:, 'a'] (si on veut une Series) ou df.loc[:, ['a']] si DF
# Idem df['a'] retourne une Series, df[['a']] retourne un DataFrame 

,a
0,0
1,1
2,2
3,3


# Slices
Principe général
Détailler suivant le type d'index : entiers, labels, datetime

Pour simplifier, les exemples suivants sont donnés pour le cas d'un objet indexé par des entiers : 
* Forme générale ```a[start:end]``` : 
    * ```a[start:end]``` : retourne les valeurs des index start à end-1
    * ```a[:end]``` : retourne toutes les valeurs du début de l'index à end-1
    * ```a[start:]``` : retourne toutes les valeurs de start à la fin
    * ```a[:]``` : copie l'objet ```a```
* Les index peuvent aussi être négatifs, on part alors de la fin de l'objet : 
    * ```a[-2]``` : retourne l'avant-dernière valeur de la série
    * ```a[-2:]``` : retourne toutes les valeurs de l'avant-dernière valeur (index -2) à la fin de l'objet
    * ```a[:-2]``` : retourne toutes les valeurs du début de la série à l'avant avant-dernière (valeur d'index -2-1=-3) valeur
* Forme générale ```a[start:end:step]```. Ex : a[::2] ne renvoie que les valeurs d'index pair.
    * Les steps peuvent aussi être négatifs, la série étant en particulier parcourue de droite à gauche, des plus grands indices vers zéro. En particulier, a[::-1] inverse la série de données.

Remarque : On peut aussi utiliser le constructeur ```slice```. 
Ex :  
```python
last_nine_slice = slice(-9, None)
a[last_nine_slice]
```

Les index peuvent être d'autres types que les entiers. Les objets slice fonctionnent également, Python essayant de convertir les différents éléments définissant de la slice vers le type de l'index (notamment important si on est indexé par des dates ou des datetimes).

https://towardsdatascience.com/basic-time-series-manipulation-with-pandas-4432afee64ea

In [44]:
a = range(1,11)
a[-2]
a[:-2]
a[-2:]
a[::2]
a[::-1]

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [48]:
dft = pd.DataFrame(100000,
                   columns=['A'],
                   index=pd.date_range('20130101',periods=100000,freq='T'))

# date_range fait pour ça : renvoie un objet DatetimeIndexDatetimeIndex

dft['2013-2':'2013-3'].head()

,A
2013-02-01 00:00:00,100000
2013-02-01 00:01:00,100000
2013-02-01 00:02:00,100000
2013-02-01 00:03:00,100000
2013-02-01 00:04:00,100000


In [61]:
from datetime import timedelta, date

dfd = pd.DataFrame({'time' : [date(2018, 1,1) + timedelta(days=i) for i in range(100)], 'value' : range(100)})
# dfd.loc[dfd['time'] < '2018-01'] ne marche pas (dans cette version de pandas au moins) => il faut avoir un objet dt et non str
dfd.set_index('time')
# dfd['2013-1':'2018-2'] ne marche pas (cannot do slice indexing on <class 'pandas.core.indexes.range.RangeIndex'>)
# Il faut surement un type particulier d'Index pour que ça marche (DatetimeIndexDatetimeIndex)

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.range.RangeIndex'> with these indexers [2013-1] of <type 'str'>

#### Voir
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.at.html